In [ ]:
from langchain import OpenAI
from langchain.chains import LLMMathChain
from langchain.agents import Tool
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

# Load env variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
llm = OpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-3.5-turbo",
    temperature=0,
    
)

In [ ]:
llm_math = LLMMathChain(llm=llm)

# initialize the math tool
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)
# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]

tools[0].name, tools[0].description

In [ ]:
# There already exists prebuilt tools in Langchain (load_tools to load them)

tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [ ]:
tools[0].name, tools[0].description

In [ ]:
tools[0].func

In [ ]:
[x.name for x in tools]

In [ ]:
# ReAct agent: Agent framework where Agent will reason on prompt, decide on action and pass information to the action item (tool)
# -> Agent receives a response from the tool and repeats reasoning process and action
# Agent decides which action to take based on tool description
# Define max_iterations to avoid infinite loop of reasoning/actions
# Set verbose=False to only show final Agent output

zero_shot_agent = initialize_agent(
	agent="zero-shot-react-description", # zero-shot agent looks only at current prompt (no memory), focuses on current task only, one interaction only
	tools=tools,
	llm=llm,
	verbose=True,
	max_iterations=3
)

In [ ]:
zero_shot_agent("what is (4.5*2.1)^2.2?")

In [ ]:
zero_shot_agent("if Mary has four apples and Giorgio brings two and a half apple "
                "boxes (apple box contains eight apples), how many apples do we "
                "have?")

In [ ]:
zero_shot_agent("what is the capital of Norway?")

In [ ]:
prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

# initialize the LLM tool
llm_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description='use this tool for general purpose queries and logic'
)

In [ ]:
tools.append(llm_tool)

In [ ]:
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

In [ ]:
zero_shot_agent("what is the capital of Norway?")

# Agent

When building an Agent, we need to:

- define the tools or the toolkit
- define the llm
- define the agent type

First, we have the tools which are included in the prompt. Second we have a thought process which was before was immediate in chains but now involves a 'thought', 'action', 'action input', 'observation' sequence.

Suffice it to say for now that the LLM now has the ability to 'reason' on how to best use tools to solve our query and can combine them in intelligent ways with just a brief description of each of them. If you want to learn more about this paradigm (MRKL) in detail, please refer to this paper.

Finally, let's pay attention to the 'agent_scratchpad'. What is that? Well, that is where we will be appending every thought or action that the agent has already performed. In this way, at each point in time, the agent will know what it has found out and will be able to continue its thought process. In other words, after using a tool it adds its thoughts and observations to the scratchpad and picks up from there.


In [ ]:
from langchain import OpenAI

llm = OpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0
)

In [ ]:
from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

# Zero Shot ReAct Agent - SQL DB

In [ ]:
from sqlalchemy import MetaData
from sqlalchemy import Column, Integer, String, Table, Date, Float
from sqlalchemy import create_engine
from sqlalchemy import insert
from datetime import datetime

from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [ ]:
metadata_obj = MetaData()

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),
)

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]
     
def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0],
    stock_ticker=obs[1],
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)



for obs in observations:
    insert_obs(obs)
     
db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=3
)

In [ ]:
result = count_tokens(
    agent_executor,
    "What is the multiplication of the ratio between stock " +
    "prices for 'ABC' and 'XYZ' in January 3rd and the ratio " +
    "between the same stock prices in January the 4th?"
)

In [ ]:
print(agent_executor.agent.llm_chain.prompt.template)

# Conversational ReAct Agent

The zero shot agent is really interesting but, as we said before, it has no memory. What if we want an assistant that remembers things we have talked about and can also reason about them and use tools? For that we have the conversational react agent.

In [ ]:
from langchain.agents import load_tools

tools = load_tools(
    ["llm-math"],
    llm=llm
)

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
from langchain.agents import initialize_agent

conversational_agent = initialize_agent(
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
)

In [ ]:
result = count_tokens(
    conversational_agent,
    "What's the result of an investment of $10,000 growing at 8% annually for 5 years with compound interest?"
)

In [ ]:
print(conversational_agent.agent.llm_chain.prompt.template)

In [ ]:
result = count_tokens(
    conversational_agent,
    "If we start with $15,000 instead and follow the same 8% annual growth for 5 years with compound interest, how much more would we have compared to the previous scenario?"
)   


# React Docstore Agent

This type of agent is similar to the ones we have seen so far but it includes the interaction with a docstore. It will have two and only two tools at its disposal: 'Search' and 'Lookup'.

With 'Search' it will bring up a relevant article and with 'Lookup' the agent will find the right piece of information in the article. This is probably easiest to see in an example:

In [ ]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer, Tool

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

In [ ]:
docstore_agent = initialize_agent(
    tools,
    llm,
    agent="react-docstore",
    verbose=True,
    max_iterations=3
)

In [ ]:
count_tokens(docstore_agent, "What were Archimedes' last words?")

In [ ]:
print(docstore_agent.agent.llm_chain.prompt.template)

# Self-Ask Agent with Search

This is the first-choice agent to use when using LLM's to extract information with a search engine. The agent will ask follow-up questions and use the search functionality to get intermediate answers that help it get to a final answer.

In [1]:
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool

search = SerpAPIWrapper(serpapi_api_key='api_key')
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description='google search'
    )
]

self_ask_with_search = initialize_agent(tools, llm, agent="self-ask-with-search", verbose=True)

ValidationError: 1 validation error for SerpAPIWrapper
__root__
  Could not import serpapi python package. Please install it with `pip install google-search-results`. (type=value_error)

In [ ]:
print(self_ask_with_search.agent.llm_chain.prompt.template)